<a href="https://colab.research.google.com/github/Rhqo/LLM_Study/blob/main/cot_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Chain of Thought(Cot) test

In [4]:
!pip install datasets -q
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from tqdm import tqdm
import random
import time
import numpy as np
from datasets import load_dataset
gsm_dataset = load_dataset("gsm8k", "main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [8]:
from openai import OpenAI
openai_client = OpenAI(
    api_key = 'sk-...'
)

In [9]:
print(gsm_dataset['train']['question'][0])
print()
print(gsm_dataset['train']['answer'][0])

Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
#### 72


In [10]:
def format_k_shot_gsm(examples, cot=True):
    if cot:

        return '\n###\n'.join(
            [f'Question: {e["question"]}\nReasoning: {e["answer"].split("####")[0].strip()}\nAnswer: {e["answer"].split("#### ")[-1]}' for e in examples]
        )
    else:
        return '\n###\n'.join(
            [f'Question: {e["question"]}\nAnswer: {e["answer"].split("#### ")[-1]}' for e in examples]
        )

In [11]:
unanswered_example = gsm_dataset['test'][2]
print(unanswered_example)

PROMPT = f"""Answer the arithmetic problem in the following format:

{format_k_shot_gsm(list(gsm_dataset['train'])[:3])}
###
Question: {unanswered_example["question"]}
Reasoning:""".strip()

print(PROMPT)

{'question': 'Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?', 'answer': 'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000>>120,000\nSo the new value of the house is 120,000+80,000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200,000-130,000=$<<200000-130000=70000>>70,000\n#### 70000'}
Answer the arithmetic problem in the following format:

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Reasoning: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
Answer: 72
###
Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of 

In [12]:
def test_prompt_openai(prompt, suppress=False, model='gpt-3.5-turbo', **kwargs):
    " a simple function to take in a prompt and run it through a given non-chat model "

    chat_completion = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        **kwargs
    )
    answer = chat_completion.choices[0].message.content
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer

In [14]:
test_prompt_openai(PROMPT, model='gpt-3.5-turbo')

PROMPT:
------
Answer the arithmetic problem in the following format:

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Reasoning: Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.
Answer: 72
###
Question: Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Reasoning: Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.
Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.
Answer: 10
###
Question: Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Reasoning: In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.
Betty's grandpare

In [15]:
test_prompt_openai(
    gsm_dataset['test'][2]['question'],
    model='gpt-3.5-turbo',
    temperature=0
)

PROMPT:
------
Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?
------
RESPONSE
------
First, we need to calculate the new value of the house after the repairs.

Original value of the house: $80,000
Cost of repairs: $50,000

Total cost of the house after repairs: $80,000 + $50,000 = $130,000

The value of the house increased by 150%, so the new value of the house is 150% of $130,000:

$130,000 * 1.5 = $195,000

Now, we need to calculate the profit Josh made:

Profit = New value of the house - Total cost
Profit = $195,000 - $130,000
Profit = $65,000

Therefore, Josh made a profit of $65,000 from flipping the house.


In [16]:
def test_k_shot(
    k, gsm_datapoint, verbose=False, how='closest', cot=True,
    options=['curie', 'cohere', 'chatgpt', 'davinci', 'base-flan-t4', 'large-flan-t5']
):
    results = {}
    query_emb = model.encode(gsm_datapoint['question'])
    if k == -1:
        PROMPT = f"""Answer the arithmetic problem in the following format:
                  Question: (an arithmetic question)
                  Answer: (the final answer as a number)
                  ###
                  Question: {gsm_datapoint["question"]}""".strip()
    elif k == 0:  # we can at least give the model a format to follow
        PROMPT = f"""Answer the arithmetic problem in the following format:
                  Question: (an arithmetic question)
                  Reasoning: (thinking through step by step on how to solve the problem)
                  Answer: (the final answer as a number)
                  ###
                  Question: {gsm_datapoint["question"]}
                  Reasoning:""".strip()
    else:
        if type(k) == float:  # using a threshold
            scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
#             scores = np.where(scores < k, -1, scores)

            # Filter out examples with a score of -1
            filtered_indices = [i for i, score in enumerate(scores) if score >= k]
            if len(filtered_indices) == 0:
                print('no examples found at that threshold. Using K=3')
                k = 3
            else:
                k = len(filtered_indices)

            # Retrieve the corresponding examples from the dataset
            examples = [gsm_dataset['train'][int(_)] for _ in np.argsort(scores)[-k:][::-1]]
            if verbose:
                print(f'Using {len(examples)} examples')
        elif how == 'closest':
            scores = util.dot_score(query_emb, doc_emb)[0].cpu().tolist()
            examples = [gsm_dataset['train'][int(_)] for _ in np.argsort(scores)[-k:][::-1]]
        elif how == 'random':
            examples = random.sample(list(gsm_dataset['train']), k)
        if cot:
            PROMPT = f"""Answer the arithmetic problem in the following format:

{format_k_shot_gsm(examples, cot=cot)}
###
Question: {gsm_datapoint["question"]}
Reasoning:""".strip()
        else:
            PROMPT = f"""Answer the arithmetic problem in the following format:

{format_k_shot_gsm(examples, cot=cot)}
###
Question: {gsm_datapoint["question"]}""".strip()
    if verbose:
        print(PROMPT)

    if 'chatgpt' in options:
        results['chatgpt'] = extract_num(
            test_prompt_openai(PROMPT, model='gpt-3.5-turbo', temperature=0, suppress=True).split('Answer: ')[-1]
        )
    if 'gpt-4' in options:
        results['gpt-4'] = extract_num(
            test_prompt_openai(PROMPT, model='gpt-4', temperature=0, suppress=True).split('Answer: ')[-1]
        )
    if 'davinci' in options:
        results['davinci'] = extract_num(
            test_prompt_openai(PROMPT, model='text-davinci-003', temperature=0, suppress=True).split('Answer: ')[-1]
        )
    if 'curie' in options:
        results['curie'] = extract_num(
            test_prompt_openai(PROMPT, model='text-curie-001', temperature=0, suppress=True).split('Answer: ')[-1]
        )
    if 'cohere' in options:
        results['cohere'] = extract_num(
            test_prompt_cohere(PROMPT, temperature=0, suppress=True).split('Answer: ')[-1]
        )

    if 'base-flan-t5' in options:
        flan_response = FLAN_PROMPT(PROMPT, base_model, base_tokenizer)
        if verbose:
            print(f'Base FLAN-T5: {flan_response}')
        results['base-flan-t5'] = extract_num(flan_response.split('Answer: ')[-1])
    if 'large-flan-t5' in options:
        flan_response = FLAN_PROMPT(PROMPT, large_model, large_tokenizer)
        results['large-flan-t5'] = extract_num(flan_response.split('Answer: ')[-1])
    results['answer'] = extract_num(gsm_datapoint['answer'].split('#### ')[-1])

    return results

In [17]:
gsm_sample = random.sample(list(gsm_dataset['test']), 200)

In [19]:
closest_results = {}

error = 0

for k in tqdm([0, 1, 3, 5, 7]):
    closest_results[f'Closest K={k}'] = []
    for i, gsm in enumerate(tqdm(gsm_sample)):
        try:
            closest_results[f'Closest K={k}'].append(
                test_k_shot(
                    k, gsm, verbose=False, how='closest',
                    options=['chatgpt', 'gpt-4', 'base-flan-t5', 'large-flan-t5']
                )
            )
        except Exception as e:
            error += 1
            print(f'Error: {error}. {e}. i={i}. K={k}')
        time.sleep(10)

  0%|          | 0/200 [00:00<?, ?it/s]

Error: 1. name 'model' is not defined. i=0. K=0



  0%|          | 1/200 [00:10<33:11, 10.01s/it]

Error: 2. name 'model' is not defined. i=1. K=0


  0%|          | 0/5 [00:14<?, ?it/s]


KeyboardInterrupt: 